In [ ]:
---
title: "Developing notebook"
format: html
---

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize as opt
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import csv
import os
from scipy.stats import pearsonr
import scipy.odr as odr
import pandas as pd
import networkx as nx
import seaborn as sns


In [ ]:
fileloc = './'
matrix_w = pd.read_csv(fileloc, header=None)
print(matrix_w.shape)
# print(matrix_w.abs().max().max())
# print(matrix_w.abs().min().min())